# Lab 9 Exercises

If we consider a European option with exercise time $T$ and payoff function $p$ and assume the validity of Black-Scholes market model, then the option price  at time $t$ is given by 

$v(S(t),t)=u(\ln(S(t)),t)$, 

where $u$ is the solution to the problem


\begin{equation*}
\frac{\partial u}{\partial
  t}(x,t)+\alpha(x,t)\frac{\partial^2 u}{\partial x^2}(x,t)+
\beta(x,t)\frac{\partial u}{\partial x}(x,t)-r\, u(x,t)=0,\ x\in
\mathbb{R}, 0\leq t<T      
\end{equation*}
                      
                      
satisfying the final condition
$$u(x,T)=p(e^x),\ x\in \mathbb{R}.$$
Here
\begin{align*}
\alpha(x,t)&=\frac{\sigma^2(e^x,t)}{2},\\
\beta(x,t)&=r-D-\frac{\sigma^2(e^x,t)}{2}.
\end{align*}

                      
For solving the equation for $u$ numerically, we introduce two boundaries $x_{min}$ and $x_{max}$ and specify boundary conditions 
                      
$u(x_{min},t)=\phi_1(t,x_{min})$,

$u(x_{max},t)=\phi_2(t,x_{max})$ 
                      
at those points. 
                      
Next, we introduce the points 
                      
$x_i=x_{min}+i\Delta x,\ i=0,\ldots,n$ 

and 
                      
$t_k=k\Delta t,\ k=0,\ldots,m$ 
                      
and denote approximate values of $u(x_i,t_k)$ by $U_{ik}$. Here $\Delta x=\frac{x_{max}-x_{min}}{n}$ and $\Delta t=\frac{T}{m}$. In the case of the explicit finite difference method we compute the values $U_{ik}$ as follows:
                      
\begin{align*}
U_{im}&=p(e^{x_i}),\ i=0,\ldots,n\\
\\                      
U_{0,k-1}&=\phi_1(t_{k-1}),\ U_{n,k-1}= \phi_2(t_{k-1}),\ k=m,m-1,\ldots,1,\\
\\                      
U_{i,k-1}&=a_{ik}U_{i-1,k}+b_{ik}U_{ik}+c_{ik}U_{i+1,k},\ i=1,\ldots,n-1,\ k=m,m-1,\ldots,1,
\end{align*}
                      
where
                      
\begin{align*}
  a_{ik}&=\frac{\Delta t}{\Delta
  x^2}\left(\alpha(x_i,t_k)-\frac{\beta(x_i,t_k)}{2}\Delta x\right),\\
\\                      
 b_{ik}&=1-2\frac{\Delta t}{\Delta
  x^2}\alpha(x_i,t_k)-r\Delta t,\\
\\                      
c_{ik}&=\frac{\Delta t}{\Delta
  x^2}\left(\alpha(x_i,t_k)+\frac{\beta(x_i,t_k)}{2}\Delta x\right).
\end{align*}
                      
If $\sigma$ satisfies $0<c_1 \leq \sigma(s,t)\leq c_2\ \forall s>0,\ \forall t\in [0,T)$ for some constants $c_1$ and $c_2$, then the method is stable for large enough $m,n$ if $b_{ik}\geq 0$. This means that we can choose one of the paramters $m,n$ freely, but the other one should be chosen so that the constraint is satisfied. Usually it is better to choose $n$ and then choose $m$ so that the condition $b_{ik}\geq 0$ is satisfied.

If $\sigma$ is a constant, then the coefficients $a=a_{ik},b=b_{ik}$ and $c=c_{ik}$ are also constants (do not depend on $i$ and $k$) and the numerical scheme simplifies to 

$$U_{i,k-1}=a\,U_{i-1,k}+b\,U_{ik}+c\,U_{i+1,k},\ i=1,\ldots,n-1,\ k=m,m-1,\ldots,1.$$

The stability condition is in this case $b\geq 0$.

If we write down this condition in terms of $m$ and $n$ we get
$$1-2\frac{T}{m}\frac{n^2}{(x_{max}-x_{min})^2}\alpha-r\frac{T}{m}\geq 0,$$
where $\alpha=\frac{\sigma^2}{2}$. 

## Exercise 1.
Solve the stability condition for $m$, that is, find an equivalent condition of the form $m\geq \ldots$.

*Solution*: ???

<details>
  <summary>After finding the formula, check it (click to expand)</summary>
   
An equivalent inequality is  
$$m\geq T\cdot \left(\frac{2\,\alpha\, n^2}{(x_{max}-x_{min})^2}+r\right)$$
</details>

## Exercise 2
Write a function that for given values of $n$, $\rho>1$, $r$, $D$, $S0$, $T$, $\sigma$ and for given functions $p$, $\phi_1$ and $\phi_2$ takes $m$ to be equal to the smallest integer satisfying the stability constraint (hint: use commands `np.int64(np.ceil(...))`) and returns the values $U_{i0},\ i=0,\ldots,n$ of the approximate solution (option prices) obtained by solving the transformed BS equation with the explicit finite difference method and the corresponding stock prices $S_i=e^{x_i}$ in the case $x_{min}=\ln \frac{S0}{\rho},\ x_{max}=\ln(\rho\, S0)$. 

Test the correctness of your code by comparing the results to the exact values obtained by Black-Scholes formula in the case $r=0.03$, $\sigma=0.5$, $D=0.05$, $T=0.5$, $E=97$, $S0=100$, $p(s)=\max(s-E,0)$, $\phi_1(t)=p(e^{x_{min}}),\ \phi_2(t)=p(e^{x_{max}})$.

**Solution**

In [4]:
import numpy as np
def explicit_solver(n, rho, r, D, S0, T, sigma, p, phi1, phi2):
    """assume that phi1 and phi2 are functions of 2 arguments (xmin,t or xmax,t)"""
    
    xmin = np.log(S0/rho)
    xmax = np.log(rho*S0)
    dx = (xmax - xmin)/n
    
    # find m from the stability condition
    m = T*((sigma**2 * n**2)/(xmax - xmin)**2 + r)
    
    # change it to an integer, which is not smaller than the computed value
    m = np.int64(np.ceil(T*((sigma**2 * n**2)/(xmax - xmin)**2 + r)))
    dt = T/m
    
    # define values of x_i
    x = np.linspace(xmin, xmax, n + 1)
    
    # n+1 by m+1 matrix
    U = np.zeros(shape = (n+1, m+1))
    
    # final condition from above
    U[:, m] = p(np.exp(x))
    
    alpha = sigma**2/2
    beta = r - D - sigma**2/2
    a = dt/dx**2 * (alpha - beta/2 * dx)
    b = 1 - 2*dt/dx**2*alpha - r*dt
    c = dt/dx**2*(alpha + beta/2*dx)
    
    i = np.arange(1, n)
    t = np.linspace(0, T, m+1)
    
    for k in range(m, 0, -1): # backward iteration, k = m, m-1, ...
        # boundary conditions
        U[0, k-1] = phi1(xmin, t[k-1])
        U[n, k-1] = phi2(xmax, t[k-1])
        
        # all other values
        U[i, k-1] = a*U[i-1, k] + b*U[i, k] + c*U[i+1, k]
  
    return [U[:, 0], np.exp(x)]

Test the solution. For this, define the pay-off function and the functions $\phi_1,\ \phi_2$

In [2]:
# define parameters
r = .03
sigma = .5
D = .05
T = .5
E = 97
S0 = 100

# define payoff function
def p_call(s):
    return np.maximum(0, s - E)
# define boundary functions
def phi1(xmin, t):
    return p_call(np.exp(xmin))
def phi2(xmax, t):
    return p_call(np.exp(xmax))
n = 6; rho = 4
solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
print("option prices:", solution[0])
print("stock prices:", solution[1])

option prices: [  0.           0.           0.64273791  14.42641365  58.99043012
 149.76470435 303.        ]
stock prices: [ 25.          39.6850263   62.99605249 100.         158.7401052
 251.98420998 400.        ]


<details>
<summary> Click to check your results!</summary>

option prices: [   0.            0.            0.64273791   14.42641365   58.99043012
  149.76470435  303.        ]<br/>
stock prices: [  25.           39.6850263    62.99605249  100.          158.7401052
  251.98420998  400.        ]
 </details>

Note that the relation to the matrix U computed inside the function `explicit_solver` and the option prices are as follows:
if at the time moment $t=t_k$ the logarithm of the current stock price $\ln(S(t))$ is $x_i$, then the price is approximately $U_{ik}$. Since we return only to values corresponding to $t=t_0=0$, the first vector of the result gives the prices for different possible values of $S(0)$: if $\ln(S(0))=x_i$, then the option price is the $i$-th value in the first vector.
So $x_{min},\ x_{max}$ and $n$ determine for which stock prices we have approximate values in the matrix $U$ (and thus also in the first component of the result of the function). Now, if we know in advance that we are mostly interested in the case  $S(0)=S_0$, where $S_0$ is a given number, it is good to make sure that $\ln S_0$ is one of the grid points in $x$ direction.

This is why we have defined $x_{min}$ and $x_{max}$ by the formulas given in the previous exercise handout. Namely, since $\frac{x_{min}+x_{max}}{2}=\ln S_0$ (show it!), we have the price corresponding to $S(0)=S_0$ in the matrix $U$ whenever $\frac{x_{min}+x_{max}}{2}$ is a grid point, and that is true for all even values of $n$. Moreover, the price is exactly in the middle of the $n+1$ values our function returns, which is clearly seen also by looking at the stock prices in the output: the value $S_0=100$ is exactly the middle element of the vector of stock prices.

Compare the approximate price of the option to the exact value computed by the BS formula:

In [3]:
import numpy as np 
from scipy import stats #needed for Phi

phi = stats.norm.cdf # the cumulative distribution function of the standard normal distribution

# define Put and Call functions
def call(S, E, T, r, sigma, D):
    d1 = (np.log(S/E) + (r - D + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    answer = S*np.exp(-D*T)*phi(d1) - E*np.exp(-r*T)*phi(d2)
    return(answer)

def put(S, E, T, r, sigma, D):
    d1 = (np.log(S/E) + (r - D + sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    answer = -S*np.exp(-D*T)*phi(-d1) + E*np.exp(-r*T)*phi(-d2)
    return(answer)

n = 6; rho = 4

approximate_solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
approximate_call_price = approximate_solution[0][n//2]

exact_call_price = call(S0, E, T, r, sigma, D)

print(exact_call_price - approximate_call_price)

0.13019861406394106


<details>
    <summary> Click to check your result!</summary>
  
  The answer should be 0.130198614064
  </details>    

The difference is quite small even for $n=6$. Check the result for a larger value of $n$:

In [4]:
n = 100

approximate_solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
approximate_call_price_2 = approximate_solution[0][n//2]

print(exact_call_price - approximate_call_price_2)

0.00415215498838073


<details>
<summary> Click to check your result!</summary>

The answer should be 0.00415215498838
</details>

## Exercise 3
Let $r=0.02$, $\sigma=0.6$, $D=0.03$, $T=0.5$, $E=99$, $S0=100$, $p(s)=\max(s-E,0)$. If we use the explicit method of previous exercise, then even if we let $n$ go to infinity there is going to be a finite error between the exact option price at $t=0, S(0)=S0$ and the corresponding approximate value. This error is caused by introducing artificial boundaries $x_{min}$ and $x_{max}$ and the boundary conditions specified at those boundaries. Use the boundary conditions $\phi_1(t)=p(e^{x_{min}}),\ \phi_2(t)=p(e^{x_{max}})$ and determine the value of the resulting error for $\rho=1.5, 2, 2.5$. In order to see the resulting error you should do several computations with fixed $\rho$ and increasing values of $n$ (assuming $m$ is determined from the stability condition, $n$ should be increased by multiplying it by 2 each time). Use the knowledge that for large enough $n$ the part of the error depending on the choice of $n$ behaves approximately like $\frac{const.}{n^2}$ (so the difference of the last two computations divided by 3 is an estimate of this part of the error for the last computation) for determining how far your last computation is from the limiting value. For each value of $\rho$, start computations from $n=10$ and stop the computations when the absolute error depending on $n$ is at least 10 times smaller than the actual absolute error of the computed answer.

**Solution**

In [5]:
# define parameters, compute the exact price
r = .02
sigma = .6
D = .03
T = .5
E = 99
S0 = 100

exact_call_price = call(S0, E, T, r, sigma, D)

First value of $\rho$:

In [6]:
rho = 1.5
n = 10

approximate_solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
approximate_price_1 = approximate_solution[0][n//2]

print(approximate_price_1 - exact_call_price)

n = 2*n

approximate_solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
approximate_price_2 = approximate_solution[0][n//2]

print(approximate_price_2 - exact_call_price)

print("doubling n changes the answer by", np.abs(approximate_price_2 - approximate_price_1)/3)

-0.6614981281249541
-0.6963219137351828
doubling n changes the answer by 0.011607928536742898


<details>
    <summary>Click to check your results!</summary>
    
    The required accuracy is achieved with n=20, the the actual absolute eror is 0.696321913735, the estimate for the error depending on n is 0.0116079285367
</details>

The second value of $\rho$:

In [7]:
rho = 2
n = 10

approximate_solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
approximate_price_1 = approximate_solution[0][n//2]

print(approximate_price_1 - exact_call_price)

n = 2*n

approximate_solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
approximate_price_2 = approximate_solution[0][n//2]

print(approximate_price_2 - exact_call_price)

print("doubling n changes the answer by", np.abs(approximate_price_2 - approximate_price_1)/3)

-0.015281464246797327
0.03175560207778716
doubling n changes the answer by 0.01567902210819483


<details>
    <summary>Click to check your results!</summary>
    
    The required accuracy is achieved with n=40, the the actual absolute eror is 0.0343809095772, the estimate for the error depending on n is 0.000875102499802
</details>

So the limiting error is approximately 0.034

The last value of $\rho$:

In [8]:
rho = 2.5
n = 10

approximate_solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
approximate_price_2 = approximate_solution[0][n//2]

change = 1
error = 1

while(change/np.abs(error) > .1):
    approximate_price_1 = approximate_price_2
    n = 2*n
    approximate_solution = explicit_solver(n, rho, r, D, S0, T, sigma, p_call, phi1, phi2)
    approximate_price_2 = approximate_solution[0][n//2]
    error = approximate_price_2 - exact_call_price
    change = np.abs((approximate_price_2 - approximate_price_1)/3)
    print(change)
print("doubling n changes the answer by", np.abs(approximate_price_2 - approximate_price_1)/3)

0.033609349997795114
0.005018937123310252
0.002348482547988103
0.0017005496556983246
6.427908793090371e-05
doubling n changes the answer by 6.427908793090371e-05


<details>
    <summary>Click to check your results!</summary>
    
    The required accuracy is achieved with n=320, the the actual absolute eror is 0.012070992458767194, the estimate for the error depending on n is 0.00006.427908793090371
</details>

So we see that the error caused by the location of $x_{min}$ and $x_{max}$ is approaching 0 quickly when $\rho$ increases. For most practical problems $\rho=3$ is good enough but of cause we should do some computations to check if this is indeed the case.

###### Practical homework 4
It is important to understand that a single computation by a finite difference method can be used for computing prices of the same option at different time moments for any reasonable value of the current stock price at the time moments of interest.  Of cause our numerical method does not give us a formula for the function $v$, but only a table of values corresponding to certain stock prices and time moments. So to find a value of the function for a given value of $s$ and for given time moment, we should determine the closest values of of stock prices and time moments for which we have results and to use them to compute the value we need. One commonly used approach is **bilinear interpolation**, which works as follows.

Suppose that we know the values of a function $u(x,t)$ at the corners of a rectangle $[x_1,x_2]\times [t_1,t_2]$, denote the known values by  $u(x_1,t_1)=U_{11}$, $u(x_1,t_2)=U_{12}$, $u(x_2,t_1)=U_{21}$, $u(x_2,t_2)=U_{22}$. Denote also $\Delta x=x_2-x_1,\Delta t=t_2-t_1$. Then the the value of $u$ at the point $(x,t)$, where $x_1\leq x<x_1$, $t_1\leq t\leq t_2$ can be computed approximately by the bilinear interpolation formula
$$u(x,t)\approx \frac{U_{11}(x_2-x)(t_2-t)+U_{12}(x_2-x)(t-t_1)+U_{21}(x-x_1)(t_2-t)+U_{22}(x-x_1)(t-t_1)}{\Delta x\,\Delta t}.$$
It is known that if the function $u$ has continuous second derivatives in $x$ and $t$ variables, then the approximation error is bounded by $const. \cdot (\Delta x^2+\Delta t^2)$.

### Homework 4 (deadline April 20, 2020) 
Modify the solver of Exercise 1 so that instead of $S0$ and $\rho$ it takes values of $x_{min}$ and $x_{max}$ as parameters and returns the matrix $U$ as the answer. Use the explicit finite difference method with  $r=0.02$, $\sigma=0.5$, $D=0$, $T=0.7$, $p(s)=\frac{40}{1+0.2(s-70)^2}$, $\phi_1(t,x_{min})=p(e^{x_{min}}),\ \phi_2(t,x_{max})=p(e^{x_{max}})$, $x_{min}=\ln 30,\ x_{max}=\ln 240$, $n=60$ to compute the matrix $U$. The matrix should be computed only once when solving the homework problem. Use this matrix to find approximate option prices:

1. at time $t=0$, when $S(0)=120$ without interpolation (the value is in the matrix $U$, you have to compute the indexes exactly),
2. Use $U$ to compute approximately the value of the option at time $t=0.135$, when $S(0.135)=83.79$ by using the bilinear interpolation formula.

**I ask you to solve the problem by yourself (you can ask for help from me)!** 


In [2]:
import numpy as np

r = .02
sigma = .5
D = 0
T = .7
xmin = np.log(30)
xmax = np.log(240)
n = 60

def p(s):
    return 40/(1 + .2*(s - 70)**2) 

def phi1(xmin, t):
    return p(np.exp(xmin))

def phi2(xmax, t):
    return p(np.exp(xmax))


def explicit_solver_Bill(xmin, xmax, n, r, D, T, sigma, p, phi1, phi2):
    
    dx = (xmax - xmin)/n
    
    # from the stability condition
    m = T*((sigma**2 * n**2)/(xmax - xmin)**2 + r)
    
    # change m to an integer, at least as big as the computed value
    m = np.int64(np.ceil(T*((sigma**2 * n**2)/(xmax - xmin)**2 + r)))
    dt = T/m
    
    # define values of x_i
    x = np.linspace(xmin, xmax, n + 1)
    
    # n+1 by m+1 matrix
    U = np.zeros(shape = (n+1, m+1))
    
    # final condition from above
    U[:, m] = p(np.exp(x))
    
    alpha = sigma**2/2
    beta = r - D - sigma**2/2
    a = dt/dx**2 * (alpha - beta/2 * dx)
    b = 1 - 2*dt/dx**2*alpha - r*dt
    c = dt/dx**2*(alpha + beta/2*dx)
    
    # time steps
    i = np.arange(1, n)
    
    # time vector
    t = np.linspace(0, T, m+1)
    
    for k in range(m, 0, -1): # backward iteration, k = m, m-1, ...
        
        # boundary conditions
        U[0, k-1] = phi1(xmin, t[k-1])
        U[n, k-1] = phi2(xmax, t[k-1])
        
        # all other values
        U[i, k-1] = a*U[i-1, k] + b*U[i, k] + c*U[i+1, k]
  
    return U

approximate_solution = explicit_solver_Bill(xmin, xmax, n, r, D, T, sigma, p, phi1, phi2)

dx = (xmax - xmin)/n

In [3]:
# Convert exact index to an integer
index_t0 = np.int64(round((np.log(120) - xmin)/dx)) 

print("For t = 0 and S0 = 120, the approximate value of the option is: ",approximate_solution[index_t0, 0], ".")

For t = 0 and S0 = 120, the approximate value of the option is:  1.9418822195077219 .


In [4]:
# Compute the approximate value of the option at time 𝑡 = .135, when 𝑆(0.135) = 83.79 using the bilinear interpolation formula.


# Find the endpoints of the grid the point lies between

m = np.int64(np.ceil(T*((sigma*n)**2/(xmax - xmin)**2 + r)))
dt = T/m
index_t = .135/dt 
index_x = ((np.log(83.79) - xmin)/dx)

# index of the first corner
U11 = approximate_solution[np.int64(np.floor(index_x)), np.int64(np.floor(index_t))]

# index of the second corner
U12 = approximate_solution[np.int64(np.floor(index_x)), np.int64(np.ceil(index_t))]

# index of the third corner
U21 = approximate_solution[np.int64(np.ceil(index_x)), np.int64(np.floor(index_t))]

# index of the fourth corner
U22 = approximate_solution[np.int64(np.ceil(index_x)), np.int64(np.ceil(index_t))]

# dt = dx = 1, so are omitted
option_price = (U11*(np.ceil(index_x) - index_x)*(np.ceil(index_t) - index_t) + 
                U12*(np.ceil(index_x) - index_x)*(index_t - np.floor(index_t)) + 
                U21*(index_x - np.floor(index_x))*(np.ceil(index_t) - index_t) + 
                U22*(index_x - np.floor(index_x))*(index_t - np.floor(index_t)))

print("The approximate value of the option when t = .135 and S(.135) = 83.79 is: ", option_price)

The approximate value of the option when t = .135 and S(.135) = 83.79 is:  3.7159731535038074


In [5]:
# Compute the approximate value of the option at time 𝑡 = .135, when 𝑆(0.135) = 83.79 using the bilinear interpolation formula.


# Find the endpoints of the grid the point lies between

m = np.int64(np.ceil(T*((sigma*n)**2/(xmax - xmin)**2 + r)))
dt = T/m
index_t = .135/dt 
index_x = ((np.log(83.79) - xmin)/dx)

# index of the first corner
U11 = approximate_solution[np.int64(np.floor(index_x)), np.int64(np.floor(index_t))]

# index of the second corner
U12 = approximate_solution[np.int64(np.floor(index_x)), np.int64(np.floor(index_t) + 1)]

# index of the third corner
U21 = approximate_solution[np.int64(np.floor(index_x) + 1), np.int64(np.floor(index_t))]

# index of the fourth corner
U22 = approximate_solution[np.int64(np.floor(index_x) + 1), np.int64(np.floor(index_t) + 1)]

# dt = dx = 1, so are omitted
option_price = (U11*(np.floor(index_x) + 1 - index_x)*(np.floor(index_t) + 1 - index_t) + 
                U12*(np.floor(index_x) + 1 - index_x)*(index_t - np.floor(index_t)) + 
                U21*(index_x - np.floor(index_x))*(np.floor(index_t) + 1 - index_t) + 
                U22*(index_x - np.floor(index_x))*(index_t - np.floor(index_t)))

print("The approximate value of the option when t = .135 and S(.135) = 83.79 is: ", option_price)

The approximate value of the option when t = .135 and S(.135) = 83.79 is:  3.7159731535038074
